In [25]:
import openai
# 设置API密钥和自定义域名
openai.base_url = "https://api.deepseek.com/v1/"
openai.api_key = "sk-4de30bd17dd14025be5f1eb9166cd4ad"

In [26]:
from datasets import load_dataset
test_dataset = load_dataset("glue", "sst2", split="test")

In [27]:
len(test_dataset)

1821

In [28]:
# 自定义prompt模板
def build_prompt(text):
    return f"""
Below is an instruction that describes a task,
paired with an input that provides further context. Write a
response that appropriately completes the request.
Before answering, think carefully about the question and
create a step-by-step chain of thoughts to ensure a logical
and accurate response.

### Instruction:
Analyze the given online review text and determine the sentiment polarity.
Return only a single number: 0 for negative sentiment or 1 for positive sentiment.
Do not include any additional text, explanations, or symbols. If you do not return exactly '0' or '1', the output will be considered invalid.

### Input:
{text}

### Response:"""

In [29]:
# # 以测试集中的第一个样本为例
# sample_text = test_dataset[0]['sentence']
# test_dataset[0]

{'sentence': 'uneasy mishmash of styles and genres .', 'label': -1, 'idx': 0}

In [30]:
# # 以测试集中的第一个样本为例
# sample_text = test_dataset[0]['sentence']
# prompt = build_prompt(sample_text)

In [31]:
# response = openai.chat.completions.create(
#     model="deepseek-reasoner",
#     messages=[
#         {"role": "user", "content": prompt}
#     ],
#     max_tokens=500,
#     temperature=0
# )
# response

ChatCompletion(id='36c07eb8-47f1-454b-9085-9ca295ee2d5a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='0', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, reasoning_content='Okay, let\'s see. I need to figure out the sentiment polarity of the review text "uneasy mishmash of styles and genres." The output should be either 0 for negative or 1 for positive. Alright, first, let\'s break down the sentence.\n\nThe key words here are "uneasy" and "mishmash." "Uneasy" usually has a negative connotation, implying discomfort or something not quite right. Then "mishmash" means a confused mixture, which isn\'t a positive description. When someone describes a creative work as a mishmash, especially with "uneasy," they\'re probably criticizing it for being disjointed or poorly blended. \n\nSo putting those together, the reviewer is saying that the combination of styles and genres isn\'t working well. That sounds

In [32]:
# response.choices[0].message.reasoning_content

'Okay, let\'s see. I need to figure out the sentiment polarity of the review text "uneasy mishmash of styles and genres." The output should be either 0 for negative or 1 for positive. Alright, first, let\'s break down the sentence.\n\nThe key words here are "uneasy" and "mishmash." "Uneasy" usually has a negative connotation, implying discomfort or something not quite right. Then "mishmash" means a confused mixture, which isn\'t a positive description. When someone describes a creative work as a mishmash, especially with "uneasy," they\'re probably criticizing it for being disjointed or poorly blended. \n\nSo putting those together, the reviewer is saying that the combination of styles and genres isn\'t working well. That sounds negative. Therefore, the sentiment polarity should be 0.'

In [20]:
# import re
#
# raw_output = response.choices[0].message.content
# matches = re.findall(r'(-1|1)', raw_output)
# result = matches[-1] if matches else None
# result

In [34]:
import re
import time
import random

def get_prediction(prompt, max_retries=5):
    """使用二进制指数退避算法进行 API 重试，确保返回 0 或 1"""
    base_delay = 1  # 基础等待时间（秒）
    max_delay = 60  # 最大等待时间（秒）
    attempt = 0

    while attempt <= max_retries:
        try:
            response = openai.chat.completions.create(
                model="deepseek-reasoner",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,  # 限制输出字符数，避免生成无关内容
                temperature=0
            )

            # 解析输出
            raw_output = response.choices[0].message.content.strip()

            # 使用正则表达式匹配最后一个 '0' 或 '1'
            matches = re.findall(r'\b[01]\b', raw_output)  # 只匹配单独的 0 或 1
            print(f"raw_output: {raw_output}; matched labels: {matches}")

            if matches:
                return int(matches[-1])  # 返回最后一个匹配到的 0 或 1

            raise ValueError(f"输出格式不符合要求，未找到 0 或 1: {raw_output}")

        except Exception as e:
            if attempt == max_retries:
                raise RuntimeError(f"API 调用失败 {max_retries} 次，最终错误: {str(e)}")

            # 计算二进制指数退避时间
            delay = min(base_delay * (2 ** attempt), max_delay)
            jitter = random.uniform(0.5, 1.5)  # 添加抖动，防止过载
            sleep_time = delay * jitter

            print(f"尝试 {attempt+1}/{max_retries} 失败，等待 {sleep_time:.2f}s: {str(e)}")
            time.sleep(sleep_time)
            attempt += 1


In [35]:
def process_dataset(dataset):
    """处理数据集，调用 API 获取预测结果"""
    predictions = []
    for idx,sample in enumerate(dataset):
        sample_text = sample['sentence']
        prompt = build_prompt(sample_text)
        print(f"当前处理{idx}")
        predicted_label = get_prediction(prompt)
        predictions.append((sample['idx'], predicted_label))
    return predictions

In [36]:
import zipfile
import pandas as pd

def save_submission(predictions, filename="submission.zip"):
    """保存预测结果为 GLUE 评测要求的 TSV 格式，并打包 ZIP 文件"""
    submission = pd.DataFrame(predictions, columns=["id", "label"])
    submission.to_csv("SST-2.tsv", sep="\t", index=False)

    with zipfile.ZipFile(filename, "w") as zipf:
        zipf.write("SST-2.tsv", arcname="SST-2.tsv")

    print(f"提交文件已生成并压缩为 {filename}")

In [ ]:
    # 处理测试集
predictions = process_dataset(test_dataset)
# 保存预测结果
save_submission(predictions)

当前处理0
raw_output: 0; matched labels: ['0']
当前处理1
raw_output: 0; matched labels: ['0']
当前处理2
